In [1]:
#Table doubling

class Node:
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.next_node = None
        
    def insert(self, key, value):
        if self.next_node==None:
            self.next_node = Node(key, value)
            return self.next_node
        else:
            return self.next_node.insert(key, value)
        
    def get_line(self):
        if self.next_node is None:
            return [(self.key, self.value)]
        else:
            return [(self.key, self.value)]+self.next_node.get_line()
        
class AdvancedHash():
    def __init__(self):
        self.max_table = 1
        self.count = 0
        self.table = [None for _ in range(self.max_table)]
        self.h = self.__new_hash_function(self.max_table)
        
    def histogram(self):
        return [element.get_line() if element is not None else [None] for element in self.table]
    
    def insert(self,key,value):
        index = self.h(key)
        if self.table[index]==None:
            self.table[index] = Node(key, value)
            node = self.table[index]
        else:
            node = self.table[index].insert(key, value)
        self.count+=1
        if self.count > self.max_table:
            self.__grow()
        return node
        
    def find_w_key(self,key):
        index = self.h(key)
        itr = self.table[index]
        while itr is not None:
            if itr.key == key:
                return itr.value
            else:
                itr = itr.next_node
        
    def __grow(self):
        print('>> Growing table... ',self.max_table,'->',2*self.max_table)
        old_table= self.table
        self.max_table = 2*self.max_table
        self.table = [None for _ in range(self.max_table)]
        self.h = self.__new_hash_function(self.max_table)
        self.count = 0
        for element in old_table:
            node = element
            while node is not None:
                self.insert(node.key, node.value)
                node = node.next_node
        
    def __prehash(self,key):
        if type(key)==str:
            ans = 0
            base = 256
            for c in key:
                ans = base*ans + ord(c)
            return ans
    
    
    def __new_hash_function(self, max_size):
        def h(key):
            return self.__prehash(key)%max_size
        return h

In [2]:
def print_histogram(hist):
    for row in hist:
        print(row)
    print('==================\n')
        
my_hash = AdvancedHash()

my_hash.insert("apple","사과")
print_histogram(my_hash.histogram())
my_hash.insert("pear","배")
print_histogram(my_hash.histogram())
my_hash.insert("peach","복숭아")
print_histogram(my_hash.histogram())
my_hash.insert("watermelon","수박")
print_histogram(my_hash.histogram())
my_hash.insert("strawberry","딸기")
print_histogram(my_hash.histogram())
my_hash.insert("cherry","체리")
print_histogram(my_hash.histogram())
my_hash.insert("carrot","당근")
#print_histogram(my_hash.histogram())
my_hash.insert("cucumber","오이")
print_histogram(my_hash.histogram())

[('apple', '사과')]

>> Growing table...  1 -> 2
[('pear', '배')]
[('apple', '사과')]

>> Growing table...  2 -> 4
[('peach', '복숭아')]
[('apple', '사과')]
[('pear', '배')]
[None]

[('peach', '복숭아')]
[('apple', '사과')]
[('pear', '배'), ('watermelon', '수박')]
[None]

>> Growing table...  4 -> 8
[('peach', '복숭아')]
[('strawberry', '딸기')]
[('pear', '배')]
[None]
[None]
[('apple', '사과')]
[('watermelon', '수박')]
[None]

[('peach', '복숭아')]
[('strawberry', '딸기'), ('cherry', '체리')]
[('pear', '배')]
[None]
[None]
[('apple', '사과')]
[('watermelon', '수박')]
[None]

[('peach', '복숭아')]
[('strawberry', '딸기'), ('cherry', '체리')]
[('pear', '배'), ('cucumber', '오이')]
[None]
[('carrot', '당근')]
[('apple', '사과')]
[('watermelon', '수박')]
[None]



In [3]:
# Karp-Rabin

def get_prime_greater_than(num):
    li = []
    li.append(2)
    n = 3
    while True:
        check = 1
        for p in li:
            check = check&int(n%p!=0)
            if check==0:
                break
        if check==1:
            if n>num:
                return n
            li.append(n)
        n+=2
        
class RollingHash():
    def __init__(self):
        self.word = ''
        self.p = 1
        self.hash_value=0
        self.base = 256
    
    def get_hash_value(self):
        return self.hash_value
    
    def append(self, c):
        self.word += c
        if self.p < len(self.word):
            self.__update_p()
        self.hash_value = (self.hash_value*self.base + ord(c)) % self.p
        return self.hash_value
        
    def skip(self,c):
        self.word = self.word[1:]
        self.hash_value = (self.hash_value - ord(c)*self.base**(len(self.word)-1)) % self.p
        return self.hash_value
        
    def __update_p(self):
        self.p = get_prime_greater_than(len(self.word))
        return self.p
        
    def __multidigit(self,s):
        ans = 0
        base = 256
        for c in s:
            ans = base*ans + ord(c)
        return ans

    def __str__(self):
        return str(self.word)+'\t'+str(self.p)+'\t'
    

In [4]:
def is_exactly_same_string(s1,s2):
    if len(s1) != len(s2):
        return False
    ans = True
    for c1, c2 in list(zip(s1,s2)):
        if c1!=c2:
            ans = False
            break
    return ans

def find_substring(substring, container):
    h1 = RollingHash()
    h2 = RollingHash()
    for c in substring:
        h1.append(c)
    for c in container[:len(substring)]:
        h2.append(c)
    
    if h1.get_hash_value() == h2.get_hash_value():
        if is_exactly_same_string(my_word, h2.word):
            return True
    for idx in range(0,len(container)-len(substring)):
        h2.skip(container[idx])
        h2.append(container[idx+len(substring)])
        if h1.get_hash_value() == h2.get_hash_value():
            if is_exactly_same_string(my_word, h2.word):
                return True
    return False

h1 = RollingHash()
h2 = RollingHash()

my_word = 'happy'
sentence = 'onetwothreefourgreatfivesixseveneightninetenbadeleventwelvethirteenfourteenhappyfifteensixteenseventeeneighteennineteensad'

if find_substring('happy', sentence) == True:
    print('word \'happy\' exists in the sentence.')
else:
    print('cannot find word \'happy\'.')
if find_substring('good', sentence) == True:
    print('word \'good\' exists in the sentence.')
else:
    print('cannot find word \'good\'.')

word 'happy' exists in the sentence.
cannot find word 'good'.
